In [1]:
import pandas as pd
import numpy as np
import pickle
from faker import Faker

# Initialize Faker
fake = Faker()

admission_types = [
    'URGENT', 'ELECTIVE', 'EW EMER.', 'DIRECT EMER.', 'EU OBSERVATION',
    'OBSERVATION ADMIT', 'DIRECT OBSERVATION',
    'AMBULATORY OBSERVATION', 'SURGICAL SAME DAY ADMISSION'
]
admission_locations = [
    'TRANSFER FROM HOSPITAL', 'TRANSFER FROM SKILLED NURSING FACILITY',
    'INTERNAL TRANSFER TO OR FROM PSYCH', 'PHYSICIAN REFERRAL',
    'EMERGENCY ROOM', 'PACU', 'PROCEDURE SITE',
    'WALK-IN/SELF REFERRAL', 'INFORMATION NOT AVAILABLE',
    'CLINIC REFERRAL'
]
insurance_types = [
    'Medicaid', 'Medicare', 'Other'
]
languages = [
    'ENGLISH', '?'
]
marital_statuses = [
    'SINGLE', 'MARRIED', 'DIVORCED', 'WIDOWED'
]
races = [
    'BLACK/CAPE VERDEAN', 'HISPANIC/LATINO - PUERTO RICAN', 'WHITE',
    'UNKNOWN', 'OTHER', 'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - SALVADORAN', 'UNABLE TO OBTAIN',
    'WHITE - OTHER EUROPEAN', 'PORTUGUESE', 'HISPANIC/LATINO - CUBAN',
    'PATIENT DECLINED TO ANSWER', 'WHITE - BRAZILIAN',
    'HISPANIC OR LATINO'
]

drugs = [
    'Midodrine', 'Multivitamins W/minerals', 'Sodium Chloride',
    'Nephrocaps', 'Nicotine Polacrilex', 'OLANZapine', 'Ondansetron',
    'OxyCODONE (Immediate Release)'
]

diagnoses_types = [
    'Urinary tract infection, site not specified',
    'Acute respiratory failure',
    'Asthma, unspecified type, unspecified',
    'Solitary pulmonary nodule',
    'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group D [Enterococcus]',
    'Dysphagia, oropharyngeal phase', 'Esophageal reflux', 'Hypoxemia',
    'Methicillin resistant pneumonia due to Staphylococcus aureus',
    'Attention deficit disorder with hyperactivity'
]

def generate_patients(n):
    patients = []
    for i in range(n):
        gender = np.random.choice(['M', 'F'])
        patients.append({
            'patient_id': i,
            'name': fake.name_male() if gender == "M" else fake.name_female(),
            'age': np.random.randint(1, 90),
            'gender': gender,
            'insurance': np.random.choice(insurance_types),
            'language': np.random.choice(languages),
            'maritalStatus': np.random.choice(marital_statuses),
            'race': np.random.choice(races),
        })
    return pd.DataFrame(patients)

def generate_staff(n):
    staff = []
    for i in range(n):
        staff.append({
            'staff_id': i,
            'name': fake.name(),
            'role': np.random.choice(['Physician', 'Nurse', 'Admin']),
            'shift_start': fake.time(),
            'shift_end': fake.time()
        })
    return pd.DataFrame(staff)

def generate_admissions(patients:pd.DataFrame):
    admissions = []
    for i in range(len(patients)):
        admissions.append({
            'adm_id': i,
            'patient_id': patients.iloc[i].patient_id,
            'admit_time': fake.date_time(),
            'discharge_time': None,
            'type': np.random.choice(admission_types),
            'location': np.random.choice(admission_locations),
            
        })
    return pd.DataFrame(admissions)

def generate_omr(admissions):
    table = []
    for i in range(len(admissions)):
        omr = {
            'omr_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'weight': np.random.randint(60, 120),
            ##'height': np.random.randint(150, 220)/ 10.0, #not used in modeling
            'bp_systolic': np.random.randint(80, 220),
            'bp_diastolic': np.random.randint(60, 120),
        }
        table.append(omr)
    return pd.DataFrame(table)

def generate_prescriptions(admissions):
    prescriptions = []
    for i in range(len(admissions)):
        prescriptions.append({
            'prescription_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'drug': np.random.choice(drugs),
        })
    return pd.DataFrame(prescriptions)

def generate_diagnoses(admissions:pd.DataFrame):
    diag = []
    for i in range(len(admissions)):
        diag.append({
            'diagnosis_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'diagnosis': np.random.choice(diagnoses_types),
        })
    return pd.DataFrame(diag)

def generate_rooms(n):
    rooms = []
    beds = []
    bed_taken = 0
    room_id = 0
    bed_id = 0
    for i in range(n):
        if i % 2 == 0:#create new room
            rooms.append({
                'room_id': room_id,
                'capacity': 1,
            })
            beds.append({
                "bed_id": bed_id,
                "room": room_id,
                "adm_id": None
            })
            room_id += 1
        else:
            room = np.random.choice(rooms)#add bed to existing room
            room["capacity"] += 1
            beds.append({
                "bed_id": bed_id,
                "room": room["room_id"],
                "adm_id": None
            })
        bed_id += 1

    return pd.DataFrame(rooms), pd.DataFrame(beds)

def assign_beds(admissions, beds):
    for i in range(len(admissions)):
        adm = admissions.iloc[i].adm_id
        bed = beds[beds.adm_id.isna()].sample(1).iloc[0,0]  #choose random empty bed
        beds.loc[bed, "adm_id"] = adm.astype(int)
        print(bed)

In [2]:
staff = generate_staff(10)
patients = generate_patients(10)
admissions = generate_admissions(patients)
admissions.discharge_time = pd.to_datetime(admissions.discharge_time)
omr = generate_omr(admissions)
prescriptions = generate_prescriptions(admissions)
diagnoses = generate_diagnoses(admissions)
rooms, beds = generate_rooms(20)
assign_beds(admissions, beds)

19
11
3
15
7
9
6
12
5
1


In [3]:
staff.head()

,staff_id,name,role,shift_start,shift_end
0,0,Anthony Figueroa,Physician,23:14:29,05:54:30
1,1,Hannah Mitchell DDS,Physician,09:34:32,20:33:00
2,2,Linda Santiago,Nurse,05:03:14,17:55:24
3,3,Nicholas Smith,Nurse,02:13:48,20:37:59
4,4,Anthony Robbins,Nurse,05:28:31,16:26:13


In [4]:
patients.head()

,patient_id,name,age,gender,insurance,language,maritalStatus,race
0,0,Steven Thomas PhD,37,M,Medicaid,ENGLISH,MARRIED,HISPANIC OR LATINO
1,1,Kevin Alvarado,70,M,Other,ENGLISH,DIVORCED,HISPANIC OR LATINO
2,2,Elizabeth Colon,53,F,Medicare,?,SINGLE,PORTUGUESE
3,3,Sharon Wright,20,F,Medicare,?,DIVORCED,HISPANIC OR LATINO
4,4,Gary Singleton,71,M,Other,?,DIVORCED,PATIENT DECLINED TO ANSWER


In [5]:
admissions

,adm_id,patient_id,admit_time,discharge_time,type,location
0,0,0,1980-04-21 03:23:23.260444,NaT,SURGICAL SAME DAY ADMISSION,EMERGENCY ROOM
1,1,1,2017-10-23 12:58:48.928369,NaT,EU OBSERVATION,CLINIC REFERRAL
2,2,2,2002-02-03 00:00:29.848459,NaT,AMBULATORY OBSERVATION,PROCEDURE SITE
3,3,3,1996-10-11 22:44:42.211458,NaT,DIRECT OBSERVATION,EMERGENCY ROOM
4,4,4,1974-11-04 22:50:03.253060,NaT,DIRECT OBSERVATION,PROCEDURE SITE
5,5,5,1985-07-07 03:17:00.251710,NaT,ELECTIVE,PROCEDURE SITE
6,6,6,1984-11-02 02:11:38.940359,NaT,EW EMER.,PHYSICIAN REFERRAL
7,7,7,1976-10-03 01:27:17.326160,NaT,DIRECT EMER.,TRANSFER FROM SKILLED NURSING FACILITY
8,8,8,1981-10-23 08:42:01.227729,NaT,URGENT,EMERGENCY ROOM
9,9,9,1997-01-03 11:03:40.753300,NaT,EW EMER.,TRANSFER FROM HOSPITAL


In [6]:
admissions.iloc[0,0]

0

In [7]:
omr.head()

,omr_id,adm_id,weight,bp_systolic,bp_diastolic
0,0,0,76,192,117
1,1,1,83,147,65
2,2,2,94,147,111
3,3,3,70,148,105
4,4,4,93,195,95


In [8]:
prescriptions.head()

,prescription_id,adm_id,drug
0,0,0,Nephrocaps
1,1,1,OxyCODONE (Immediate Release)
2,2,2,Nicotine Polacrilex
3,3,3,Nicotine Polacrilex
4,4,4,Midodrine


In [9]:
diagnoses.head()

,diagnosis_id,adm_id,diagnosis
0,0,0,"Dysphagia, oropharyngeal phase"
1,1,1,Streptococcus infection in conditions classifi...
2,2,2,Attention deficit disorder with hyperactivity
3,3,3,Esophageal reflux
4,4,4,Esophageal reflux


In [10]:
rooms

,room_id,capacity
0,0,5
1,1,2
2,2,1
3,3,2
4,4,4
5,5,1
6,6,2
7,7,1
8,8,1
9,9,1


In [11]:
beds

,bed_id,room,adm_id
0,0,0,None
1,1,0,9
2,2,1,None
3,3,0,2
4,4,2,None
5,5,1,8
6,6,3,6
7,7,0,4
8,8,4,None
9,9,4,5


In [13]:
with open('app/data/staff.csv', 'w') as f:
    staff.to_csv(f, index=False)
with open('app/data/patients.csv', 'w') as f:
    patients.to_csv(f, index=False)
with open('app/data/admissions.csv', 'w') as f:
    admissions.to_csv(f, index=False)
with open('app/data/omr.csv', 'w') as f:
    omr.to_csv('app/data/omr.csv', index=False)
with open('app/data/prescriptions.csv', 'w') as f:
    prescriptions.to_csv("app/data/prescriptions.csv", index=False)
with open("app/data/diagnoses.csv", "w") as f:
    diagnoses.to_csv("app/data/diagnoses.csv", index=False)
with open("app/data/rooms.csv", "w") as f:
    rooms.to_csv(f, index=False)
with open("app/data/beds.csv", "w") as f:
    beds.to_csv(f, index=False)

In [14]:
with open("app/data/admission_types.pkl", "wb") as f:
    pickle.dump(admission_types, f)
with open("app/data/admission_locations.pkl", "wb") as f:
    pickle.dump(admission_locations, f)
with open("app/data/insurance_types.pkl", "wb") as f:
    pickle.dump(insurance_types, f)
with open("app/data/languages.pkl", "wb") as f:
    pickle.dump(languages, f)
with open("app/data/marital_statuses.pkl", "wb") as f:
    pickle.dump(marital_statuses, f)
with open("app/data/races.pkl", "wb") as f:
    pickle.dump(races, f)
with open("app/data/drugs.pkl", "wb") as f:
    pickle.dump(drugs, f)
with open("app/data/diagnoses_types.pkl", "wb") as f:
    pickle.dump(diagnoses_types, f)